## Part 1 Toronto Postal codes

In [2]:
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.6.3       |           py35_0         139 KB  conda-forge

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0 conda-forge


beautifulsoup4-4.6.3 | 139 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [3]:
!conda install -c conda-forge lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libxml2-2.9.9              |       h13577e0_0         2.0 MB  conda-forge
    libxslt-1.1.32             |    h4785a14_1002         539 KB  conda-forge
    lxml-4.2.5                 |   py35hc9114bc_0         5.8 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         8.3 MB

The following packages will be UPDATED:

    libxml2: 2.9.4-h6b072ca_5     --> 2.9.9-h13577e0_0     conda-forge
    libxslt: 1.1.29-hcf9102b_5    --> 1.1.32-h4785a14_1002 conda-forge
    lxml:    4.1.0-py35ha401a81_0 --> 4.2.5-py35hc9114bc_0 conda-forge


libxml2-2.9.9        | 2.0 MB    | ##################################### | 100% 
libxslt-1.1.32

In [6]:
!conda install -c conda-forge requests --yes

Solving environment: done

# All requested packages already installed.



In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

page_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page_response = requests.get(page_link, timeout=5)
soup=BeautifulSoup(page_response.content, 'lxml')
table=soup.find('table', class_='wikitable sortable')
column_names=['Postcode', 'Borough', 'Neighbourhood']

# Determine number of rows
nrows=0
for tr in table.find_all('tr'):
    column=tr.find_all('td')
    if len(column)>0:
        nrows+=1
          
df = pd.DataFrame(columns=column_names, index=range(0,nrows))

row_marker=0
for tr in table.find_all('tr'):
    column_marker=0
    column=tr.find_all('td')
    for td in column:
        df.iat[row_marker, column_marker]=td.get_text() 
        column_marker +=1
    if len(column)>0:
        row_marker +=1

df.replace(r'\s+|\\n', ' ', regex=True, inplace=True)
df=df[df.Borough != "Not assigned"]
df=df.groupby('Postcode', as_index=False).agg(lambda x: ', '.join(set(x.dropna())))
df.Neighbourhood.replace("Not assigned ",df.Borough,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"West Hill , Morningside , Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Part 2 Geodata

In [8]:
df_g=pd.read_csv("http://cocl.us/Geospatial_data")
df_g.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Bring them together

In [21]:
df_merge = pd.merge(df, df_g,  left_on='Postcode', right_on='Postal Code')
df_merge=df_merge[["Postal Code", "Borough", "Neighbourhood", "Latitude", "Longitude"]]
df_merge

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"West Hill , Morningside , Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge , Golden Mile , Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West , Cliffside , Cliffcr...",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848
